In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pickle

In [ ]:

data = pd.read_csv('heart_failure_clinical_records_dataset (1).csv')

In [4]:
data.isnull().sum()

age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64

In [5]:
X = data.drop('DEATH_EVENT', axis=1)
y = data['DEATH_EVENT']


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [8]:
y_pred = model.predict(X_test)


In [9]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 70.00%


In [11]:
gb_model = GradientBoostingClassifier(random_state=42)


In [12]:
gb_model.fit(X_train, y_train)


GradientBoostingClassifier(random_state=42)

In [13]:
y_pred = gb_model.predict(X_test)

In [14]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy with Gradient Boosting: {accuracy * 100:.2f}%')

Accuracy with Gradient Boosting: 71.67%


In [16]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}

In [ ]:

gb_model = GradientBoostingClassifier(random_state=42)


grid_search = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)


print("Best parameters found: ", grid_search.best_params_)


best_model = grid_search.best_estimator_


y_pred = best_model.predict(X_test)


Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters found:  {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}


In [ ]:

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy after hyperparameter tuning: {accuracy * 100:.2f}%')

Accuracy after hyperparameter tuning: 73.33%


In [ ]:

xgb_model = xgb.XGBClassifier(random_state=42)


xgb_model.fit(X_train, y_train)


y_pred = xgb_model.predict(X_test)

In [ ]:

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy with XGBoost: {accuracy * 100:.2f}%')

Accuracy with XGBoost: 78.33%


In [34]:
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 2, 3]
}


xgb_grid_search = GridSearchCV(estimator=xgb.XGBClassifier(), param_grid=xgb_param_grid, cv=3, n_jobs=-1, verbose=2)
xgb_grid_search.fit(X_train, y_train)


best_xgb_model = xgb_grid_search.best_estimator_


y_pred = best_xgb_model.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy after further hyperparameter tuning for XGBoost: {accuracy * 100:.2f}%')


Fitting 3 folds for each of 324 candidates, totalling 972 fits
Accuracy after further hyperparameter tuning for XGBoost: 78.33%


In [33]:
from sklearn.model_selection import cross_val_score


cv_scores = cross_val_score(best_xgb_model, X_train, y_train, cv=5)


print(f'Cross-validation scores: {cv_scores}')
print(f'Mean cross-validation score: {cv_scores.mean() * 100:.2f}%')


Cross-validation scores: [0.875      0.89583333 0.91666667 0.8125     0.93617021]
Mean cross-validation score: 88.72%


In [35]:

with open('xgb_model.pkl', 'wb') as file:
    pickle.dump(best_xgb_model, file)
